In [1]:
# Install corenlp server by running:
#    giveme5w1h-corenlp install
# Start corenlp server by running:
#    giveme5w1h-corenlp
# corenlp does not work on new versions of Java. Install Java 8 in a conda environment with:
#    conda install -c cidermole jdk8
# Must add this caching file manually:
#    mkdir -p ~/anaconda3/envs/CONDA_ENV_NAME/lib/python3.7/site-packages/Giveme5W1H/examples/caches
#    cd ~/anaconda3/envs/CONDA_ENV_NAME/lib/python3.7/site-packages/Giveme5W1H/examples/caches
#    wget https://github.com/fhamborg/Giveme5W1H/blob/master/Giveme5W1H/examples/caches/_Nominatim.prickle?raw=true
#    mv _Nominatim.prickle\?raw\=true Nominatim.prickle
# Must download the "punkt" tokenizer models for nltk:
#    python -c "import nltk; nltk.download('punkt')"
# Must download the "en_core_web_sm" models for SpaCY:
#    https://github.com/explosion/spaCy
#    python -m spacy download en_core_web_sm
# Install dask:
#    pip install "dask[complete]" 

In [4]:
%time %run -i "/home/stevengt/Documents/code/whatwhy/whatwhy/main.py"

In [3]:
%run "/home/stevengt/Documents/code/whatwhy/notebooks/opinion parser.ipynb"

In [4]:
# df["Opinion Terms"] = df["Preprocessed Text"].apply( get_opinion_terms_from_text )

In [5]:
# pd.set_option('display.max_colwidth', -1)
# print(df[["Preprocessed Text", "Opinion Terms"]])
# df["What Raw Text"]